In [16]:
# %load imdb_data_to_graph.py
import numpy as np
import dgl
import pickle
import os
from sklearn.preprocessing import MultiLabelBinarizer
import torch
import sys
sys.path.append('../panrep/') 
import load_data
import argparse
import itertools

In [39]:
    parser = argparse.ArgumentParser(description='PanRep')
    parser.add_argument("--dropout", type=float, default=0.2,
            help="dropout probability")
    parser.add_argument("--n-hidden", type=int, default=60,
            help="number of hidden units") # use 16, 2 for debug
    parser.add_argument("--gpu", type=int, default=0,
            help="gpu")
    parser.add_argument("--lr", type=float, default=1e-2,
            help="learning rate")
    parser.add_argument("--n-bases", type=int, default=20,
            help="number of filter weight matrices, default: -1 [use all]")
    parser.add_argument("--n-layers", type=int, default=3,
            help="number of propagation rounds")
    parser.add_argument("-e", "--n-epochs", type=int, default=50,
            help="number of training epochs for decoder")
    parser.add_argument("-ec", "--n-cepochs", type=int, default=400,
                        help="number of training epochs for classification")
    parser.add_argument("-num_masked", "--n-masked-nodes", type=int, default=100,
                        help="number of masked nodes")
    parser.add_argument("-pct_masked_links", "--pct-masked-links", type=int, default=0.5,
                        help="number of masked links")
    parser.add_argument("-negative_rate", "--negative-rate", type=int, default=4,
                        help="number of negative examples per masked link")


    parser.add_argument("-d", "--dataset", type=str, required=True,
            help="dataset to use")
    parser.add_argument("-en", "--encoder", type=str, required=True,
                        help="Encoder to use")
    parser.add_argument("--l2norm", type=float, default=0.0000,
            help="l2 norm coef")
    parser.add_argument("--relabel", default=False, action='store_true',
            help="remove untouched nodes and relabel")
    parser.add_argument("--use-self-loop", default=False, action='store_true',
            help="include self feature as a special relation")
    parser.add_argument("--use-infomax-loss", default=False, action='store_true',
                        help="use infomax task supervision")
    parser.add_argument("--use-reconstruction-loss", default=True, action='store_true',
                        help="use feature reconstruction task supervision")
    parser.add_argument("--node-masking", default=False, action='store_true',
                        help="mask a subset of node features")
    parser.add_argument("--loss-over-all-nodes", default=True, action='store_true',
                        help="compute the feature reconstruction loss over all nods or just the masked")
    parser.add_argument("--link-prediction", default=False, action='store_true',
                       help="use link prediction as supervision task")
    parser.add_argument("--mask-links", default=True, action='store_true',
                       help="mask the links to be predicted")

    parser.add_argument("--batch-size", type=int, default=100,
            help="Mini-batch size. If -1, use full graph training.")
    parser.add_argument("--model_path", type=str, default=None,
            help='path for save the model')
    parser.add_argument("--fanout", type=int, default=10,
            help="Fan-out of neighbor sampling.")

    fp = parser.add_mutually_exclusive_group(required=False)
    fp.add_argument('--validation', dest='validation', action='store_true')
    fp.add_argument('--testing', dest='validation', action='store_false')
    parser.set_defaults(validation=True)

    args = parser.parse_args(['--dataset', 'imdb','--encoder', 'RGCN'])

In [40]:
train_idx, test_idx, val_idx, labels, g, category, num_classes, masked_node_types=\
        load_data.load_hetero_data(args)

Using device cuda:0
Graph(num_nodes={'movie': 1867, 'person': 13393},
      num_edges={('movie', 'directed_by', 'person'): 2378, ('movie', 'written_by', 'person'): 2704, ('movie', 'participated_by', 'person'): 10286, ('person', 'known_for', 'movie'): 4579},
      metagraph=[('movie', 'person'), ('movie', 'person'), ('movie', 'person'), ('person', 'movie')])
tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.]])


In [55]:
mapping_dict={}
edge_lists={}
reverse_mapping_dict={}
for etype in g.etypes:
    reverse_mapping_dict[etype]={}
    u,v,eid=g.all_edges(form='all', etype=etype)
    srtype,etyp,destype=g.to_canonical_etype(etype)
    u=u.data.numpy()
    v=v.data.numpy()
    u=u.astype(int)
    v=v.astype(int)
    if srtype==destype:
        uniq_v,uniq_i=np.unique((u,v),return_inverse=True)
        counter = itertools.count(1)
        # make consecutive ids
        mapping_dict[etype] = {next(counter):(v_node,srtype) for v_node in uniq_v}
        
        reverse_mapping_dict[etype][srtype]={v_node[0]:k for k,v_node in (mapping_dict[etype]).items()}
        
        edges=np.concatenate((np.expand_dims(u, axis=0),np.expand_dims(v, axis=0)),axis=0)
        #print(edges)
        edge_lists[etype]=[(reverse_mapping_dict[etype][srtype][edges[0,i]],reverse_mapping_dict[etype][destype][edges[1,i]]) for i in range((edges).shape[1])]
        #print(mapping_dict[etype][edge_lists[etype][0][1]])
        #print(v[0])
        print(counter)
        print(edge_lists[etype])
        print(max(v))
    else:
        uniq_u,uniq_i=np.unique((u),return_inverse=True)
        counter = itertools.count(1)
        # make consecutive ids
        mapping_dict[etype] = {next(counter):(u_node,srtype) for u_node in uniq_u}
        stid=next(counter)
        # continue for destination, but keep the same counter
        uniq_v,uniq_i=np.unique((v),return_inverse=True)
        for v_node in uniq_v:
            mapping_dict[etype][stid]=(v_node,destype)
            stid+=1
        #print(mapping_dict[etype])
        reverse_mapping_dict[etype][srtype]={v_node[0]:k for k,v_node in (mapping_dict[etype]).items() if v_node[1]==srtype}
        reverse_mapping_dict[etype][destype]={v_node[0]:k for k,v_node in (mapping_dict[etype]).items() if v_node[1]==destype}
        
        edges=np.concatenate((np.expand_dims(u, axis=0),np.expand_dims(v, axis=0)),axis=0)
        
        edge_lists[etype]=[(reverse_mapping_dict[etype][srtype][edges[0,i]],reverse_mapping_dict[etype][destype][edges[1,i]]) for i in range((edges).shape[1])]
        #print(mapping_dict[etype][edge_lists[etype][0][1]])
        #print(v[0])
        #print(stid)
        #print(max(v))
        
        
        

In [56]:
import pickle
folder="../data/imdb_data/"

In [57]:
f = open(folder+"mappings.pkl","wb")
pickle.dump(mapping_dict,f)
f.close()

In [60]:
for e in g.etypes:
    print(edge_lists[e])
    np.savetxt(folder+"edge_list_"+e+".mtx",edge_lists[e],fmt='%i')

[(1, 2692), (2, 3079), (3, 2115), (4, 3088), (5, 1824), (6, 3208), (7, 3471), (8, 3693), (8, 3173), (9, 2589), (10, 3469), (11, 2848), (11, 1976), (12, 3745), (13, 2316), (14, 3698), (15, 3609), (16, 1799), (17, 2820), (18, 1943), (19, 2205), (19, 3326), (20, 3467), (21, 2653), (22, 3871), (23, 2632), (24, 2088), (25, 1765), (25, 3379), (26, 3757), (27, 2119), (27, 3260), (27, 3494), (28, 3617), (29, 2120), (30, 1985), (31, 2821), (32, 2100), (33, 2996), (33, 3465), (34, 2123), (34, 2774), (35, 2683), (36, 2045), (37, 3016), (38, 3431), (39, 3590), (40, 1948), (41, 2124), (42, 3621), (43, 3556), (43, 3555), (44, 3412), (44, 3413), (45, 2948), (46, 1687), (46, 2128), (47, 2470), (48, 3703), (49, 1897), (50, 3631), (51, 3840), (52, 3037), (53, 1868), (54, 2480), (55, 3517), (56, 3008), (57, 3780), (58, 2129), (58, 3790), (59, 1952), (59, 2647), (60, 2972), (61, 2748), (61, 2541), (61, 2542), (61, 3435), (61, 2806), (61, 2149), (61, 2834), (62, 3221), (63, 3116), (63, 3124), (63, 3103), (